In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
random_state = 34

In [3]:
import json

In [4]:
import sys
sys.path.append('/home/gpuadmin/projects/candice/AGEL/agel_backend/agel_v15/copy_editing/get_grammar_suggestions')
from __parameters__ import word_tokenizer, sentence_tokenizer

In [5]:
with open('/home/gpuadmin/Candice/Data/data_prep_2.json','r',encoding = 'utf-8') as f:
    data = json.load(f)

## Prepare NER Data

In [6]:
from gensim.models import KeyedVectors
import numpy as np
import logging
from collections import Counter
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(message)s')

### Load word vectors

In [7]:
%time w2v_all_caps = KeyedVectors.load_word2vec_format("w2v_all_raw_cap.txt")

2020-01-07 12:25:29,181 loading projection weights from w2v_all_raw_cap.txt
2020-01-07 12:25:33,774 loaded (43350, 128) matrix from w2v_all_raw_cap.txt


CPU times: user 4.58 s, sys: 136 ms, total: 4.72 s
Wall time: 4.59 s


### Prepare Vocab

In [8]:
all_vocab = list(w2v_all_caps.vocab.keys())

In [9]:
all_vocab = ['<pad>'] + all_vocab + ['<unk>'] #adding padding and unknown words to vocab
print('Length of vocabulary : {}'.format(len(all_vocab)))

Length of vocabulary : 43352


#### Converting vocab to numbers and vice versa

In [10]:
vocab2int = {tok: i for i, tok in enumerate(all_vocab)}
int2vocab = {i: tok for i, tok in enumerate(all_vocab)}

def tok2int(tokens):
    ret = []
    for tok in tokens:
        temp = vocab2int[tok] if tok in vocab2int else vocab2int['<unk>']
        ret.append(temp)
    return ret

int2tok = lambda ints : [int2vocab[item] for item in ints]

### Prepare Vectors

In [11]:
all_vectors = w2v_all_caps.vectors
print('all vectors shape = {}'.format(all_vectors.shape))

all vectors shape = (43350, 128)


In [12]:
#adding vector for padding at the beginning
all_vectors = np.append(np.zeros((1,w2v_all_caps.vector_size)), all_vectors, axis=0)
#adding vector for unknown words at the end
all_vectors = np.append(all_vectors, np.random.random((1, w2v_all_caps.vector_size)), axis = 0)

print('all vectors shape = {}'.format(all_vectors.shape))

all vectors shape = (43352, 128)


### Prepare Labels

In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
all_toks = [tok for item in data for tok in item['model_output']]
label_count = Counter(all_toks)

In [15]:
label_count

Counter({'O': 621886,
         'S-–_INS': 3773,
         'S-to_INS_ONLY': 130,
         'S--_INS': 3056,
         'S-in_INS': 925,
         'S-with_INS': 5945,
         'S-to_INS': 2757,
         'S-—_INS': 1537,
         'S-of_INS': 405,
         'B-–_INS': 40,
         'I-–_INS': 7,
         'E-–_INS': 40,
         'S-of_INS_ONLY': 142,
         'S-–_INS_ONLY': 266,
         'S-because_INS': 1131,
         'S-after_INS': 1382,
         'B-to_INS': 13,
         'E-to_INS': 13,
         'B-of_INS': 9,
         'E-of_INS': 9,
         'I-to_INS': 2,
         'I-of_INS': 3})

In [16]:
all_labels = ['<pad>'] + list(label_count.keys())
print('Number of labels = {}'.format(len(all_labels)))

Number of labels = 23


In [17]:
#list of entities for NER
all_labels

['<pad>',
 'O',
 'S-–_INS',
 'S-to_INS_ONLY',
 'S--_INS',
 'S-in_INS',
 'S-with_INS',
 'S-to_INS',
 'S-—_INS',
 'S-of_INS',
 'B-–_INS',
 'I-–_INS',
 'E-–_INS',
 'S-of_INS_ONLY',
 'S-–_INS_ONLY',
 'S-because_INS',
 'S-after_INS',
 'B-to_INS',
 'E-to_INS',
 'B-of_INS',
 'E-of_INS',
 'I-to_INS',
 'I-of_INS']

#### Converting labels to integer and vice versa

In [18]:
lab2int = {lab: i for i, lab in enumerate(all_labels)}
int2lab = {i: lab for i, lab in enumerate(all_labels)}

#### one hot encode labels

In [19]:
all_labels = [[i] for i,_ in enumerate(all_labels)]

In [20]:
label_encoder = OneHotEncoder(handle_unknown='error')

In [21]:
label_encoder.fit(all_labels)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

### Prepare Training Data

In [22]:
import sys
sys.path.append('/home/gpuadmin/projects/candice/AGEL/agel_backend/agel_v15/copy_editing/get_grammar_suggestions')
from __parameters__ import word_tokenizer, sentence_tokenizer

In [23]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [24]:
dtype_float = 'float32'
dtype_int = 'int32'

#### find optimal token length

In [25]:
import matplotlib.pyplot as plt
x = [len(word_tokenizer.tokenize(item['text'])) for item in data]
plt.hist(x, 20, facecolor='g', alpha=0.75, cumulative=True, normed=True)
plt.show()

/home/gpuadmin/Candice/scripts/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: MatplotlibDeprecationWarning: 
The 'normed' kwarg was deprecated in Matplotlib 2.1 and will be removed in 3.1. Use 'density' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


<Figure size 640x480 with 1 Axes>

In [26]:
max_len = 50
min_len = 7
len([item for item in data if min_len < len(word_tokenizer.tokenize(item['text'])) < max_len])/ len(data)

0.8810531750129066

#### all padded tokens

In [27]:
%%time
sequences = [tok2int(word_tokenizer.tokenize(item['text'])) for item in data]
all_padded_toks = pad_sequences(sequences, maxlen=max_len, dtype=dtype_int, 
              padding='post',
              truncating='post',
              value=vocab2int['<pad>'])

CPU times: user 407 ms, sys: 4.31 ms, total: 412 ms
Wall time: 410 ms


In [28]:
all_padded_toks.shape

(19370, 50)

#### all padded and one hot converted labels

In [29]:
sequences = [[lab2int[ele] for ele in item['model_output']] for item in data]
all_padded_labels = pad_sequences(sequences, maxlen=max_len, dtype=dtype_int, 
              padding='post',
              truncating='post',
              value=lab2int['<pad>'])

<p> Convert to One Hot </p>

In [30]:
%time all_one_hot_labels = np.array([label_encoder.transform([[lab] for lab in padded_lab]).toarray() for padded_lab in all_padded_labels])

CPU times: user 5.4 s, sys: 81.9 ms, total: 5.49 s
Wall time: 5.48 s


In [31]:
all_one_hot_labels.shape

(19370, 50, 23)

### split train validation

In [32]:
from sklearn.model_selection import train_test_split

<p>use indices as maps for training and validation data</p>

In [33]:
val_size = 0.1
train_idxs, val_idxs, _, _ = train_test_split(
    [idx for idx, _ in enumerate(data)], [None]*len(data), test_size=val_size, random_state=random_state)
print('Length of training data = {}'.format(len(train_idxs)))
print('Length of validation data = {}'.format(len(val_idxs)))

Length of training data = 17433
Length of validation data = 1937


In [34]:
train_tokens = np.take(all_padded_toks, train_idxs, axis=0)
train_labels = np.take(all_one_hot_labels, train_idxs, axis=0)
#train_labels = np.array([all_padded_labels[idx] for idx in train_idxs])

val_tokens = np.take(all_padded_toks, val_idxs, axis=0)
val_labels = np.take(all_one_hot_labels, val_idxs, axis=0)
#val_labels = np.array([all_padded_labels[idx] for idx in val_idxs])

In [49]:
len(train_labels)

17433

## Build Model

In [35]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D
from keras.optimizers import sgd
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras.callbacks import Callback

### model parameters

In [36]:
class LearningRateScheduler(Callback):
    """Learning rate scheduler.
    # Arguments
        schedule: a function that takes an epoch index as input
            (integer, indexed from 0) and current learning rate
            and returns a new learning rate as output (float).
        verbose: int. 0: quiet, 1: update messages.
    """

    def __init__(self, schedule, verbose=0):
        super(LearningRateScheduler, self).__init__()
        self.schedule = schedule
        self.verbose = verbose
        self._losses = {'val':[], 'train': []}

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = float(K.get_value(self.model.optimizer.lr))
        try:  # new API
            lr = self.schedule(epoch, lr, self._losses)
        except TypeError:  # old API for backward compatibility
            lr = self.schedule(epoch)
        if not isinstance(lr, (float, np.float32, np.float64)):
            raise ValueError('The output of the "schedule" function '
                             'should be float.')
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: LearningRateScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        self._losses['val'].append(logs['val_loss'])
        self._losses['train'].append(logs['loss'])

In [50]:
word_embedding_dim = all_vectors.shape[1]
pre_rnn_dropout = 0.33
rnn_dropout = 0.33
rnn_units = 4
epochs = 100
lr = 0.5
momentum = 0.9
decay_rate = 0.9
decay_step = int(epochs*0.2)
cycle_step = int(epochs*0.1)
cycle_rate = 0.9

optimizer = sgd(lr=lr, momentum=momentum)

def step_decay(epoch, lr, rates):
    
    ## 0th epoch
    if not epoch:
        return lr
        
    if epoch % decay_step == 0:
        return lr * decay_rate
    
    if epoch > 50:
        ## cycle lr
        if (epoch-50) % cycle_step < cycle_step//2:
            return lr * cycle_rate
        elif (epoch-50) % cycle_step > cycle_step//2:
            return lr / cycle_rate

    return lr

lr_scheduler = LearningRateScheduler(step_decay)


In [51]:
word_in = Input(shape=(max_len,), name='token_inputs')
model = None
model = Embedding(input_dim = len(all_vocab), 
                     output_dim = word_embedding_dim,
                     input_length = max_len, 
                     mask_zero=False,
                     weights=[all_vectors],
                 name='word_embeddings', 
                 trainable = False)(word_in)
# model = SpatialDropout1D(pre_rnn_dropout, name='pre_rnn_dropout')(model)
model = LSTM(units=rnn_units, return_sequences=True,
#                                recurrent_dropout=rnn_dropout,
                               name='LSTM_cell')(model)

model = LSTM(units=rnn_units, return_sequences=True,
#                                recurrent_dropout=rnn_dropout,
                               name='LSTM_cell2')(model)

model = TimeDistributed(Dense(len(all_labels), activation="softmax"),  name='label_prediction')(model)
model = Model(inputs=[word_in], outputs=[model], name='PrepositionsNER')

#### get intermediate output

In [52]:
# from keras import backend as K
# get_output = lambda inp_, out_ :K.function([inp_], [out_])
# layer_output = get_output(model.layers[0].input, model.layers[-1].output)(val_tokens)

In [53]:
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])

In [54]:
model.summary()

Model: "PrepositionsNER"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
token_inputs (InputLayer)    (None, 50)                0         
_________________________________________________________________
word_embeddings (Embedding)  (None, 50, 128)           5549056   
_________________________________________________________________
LSTM_cell (LSTM)             (None, 50, 4)             2128      
_________________________________________________________________
LSTM_cell2 (LSTM)            (None, 50, 4)             144       
_________________________________________________________________
label_prediction (TimeDistri (None, 50, 23)            115       
Total params: 5,551,443
Trainable params: 2,387
Non-trainable params: 5,549,056
_________________________________________________________________


In [55]:
9159*5

45795

In [56]:
from keras.callbacks import TensorBoard
from time import time

In [57]:
import numpy as np
from seqeval.metrics import f1_score, classification_report, precision_score, recall_score


class F1Metrics(Callback):

    def __init__(self, id2label, pad_value=0, validation_data=None, digits=4):
        """
        Args:
            id2label (dict): id to label mapping.
            (e.g. {1: 'B-LOC', 2: 'I-LOC'})
            pad_value (int): padding value.
            digits (int or None): number of digits in printed classification report
              (use None to print only F1 score without a report).
        """
        super(F1Metrics, self).__init__()
        self.id2label = id2label
        self.pad_value = pad_value
        self.validation_data = validation_data
        self.digits = digits
        self.is_fit = validation_data is None

    def convert_idx_to_name(self, y, array_indexes):
        """Convert label index to name.
        Args:
            y (np.ndarray): label index 2d array.
            array_indexes (list): list of valid index arrays for each row.
        Returns:
            y: label name list.
        """
        y = [[self.id2label[idx] for idx in row[row_indexes]] for
             row, row_indexes in zip(y, array_indexes)]
        return y

    def predict(self, X, y):
        """Predict sequences.
        Args:
            X (np.ndarray): input data.
            y (np.ndarray): tags.
        Returns:
            y_true: true sequences.
            y_pred: predicted sequences.
        """
        y_pred = self.model.predict_on_batch(X)

        # reduce dimension.
        y_true = np.argmax(y, -1)
        y_pred = np.argmax(y_pred, -1)

        non_pad_indexes = [np.nonzero(y_true_row != self.pad_value)[0] for y_true_row in y_true]

        y_true = self.convert_idx_to_name(y_true, non_pad_indexes)
        y_pred = self.convert_idx_to_name(y_pred, non_pad_indexes)

        return y_true, y_pred

    def score(self, y_true, y_pred):
        """Calculate f1 score.
        Args:
            y_true (list): true sequences.
            y_pred (list): predicted sequences.
        Returns:
            score: f1 score.
        """
        f1 = f1_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        if self.digits:
            print(classification_report(y_true, y_pred, digits=self.digits))
        return f1, precision, recall

    def on_epoch_end(self, epoch, logs={}):
        if self.is_fit:
            self.on_epoch_end_fit(epoch, logs)
        else:
            self.on_epoch_end_fit_generator(epoch, logs)

    def on_epoch_end_fit(self, epoch, logs={}):
        X = self.validation_data[0]
        y = self.validation_data[1]
        y_true, y_pred = self.predict(X, y)
        f1, precision, recall = self.score(y_true, y_pred)
        logs['val_f1'] = f1
        logs['val_precision'] = precision
        logs['val_recall'] = recall
        #y_true, y_pred = self.predict(train_tokens, train_labels)
        #f1, prec, rec = self.score(y_true, y_pred)
        #logs['f1'] = f1
        #logs['precision'] = prec
        #logs['recall'] = rec

    def on_epoch_end_fit_generator(self, epoch, logs={}):
        y_true = []
        y_pred = []
        for X, y in self.validation_data:
            y_true_batch, y_pred_batch = self.predict(X, y)
            y_true.extend(y_true_batch)
            y_pred.extend(y_pred_batch)
        score = self.score(y_true, y_pred)
        logs['f1'] = score

In [58]:
import os
import tensorflow as tf
from keras.callbacks import TensorBoard

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [59]:
from time import gmtime, strftime

tensorboard = TrainValTensorBoard(log_dir="/home/gpuadmin/Candice/tensorboard_logs/prep_ner/{}".format(strftime("%Y-%m-%d %H:%M:%S", gmtime())))

callbacks = [F1Metrics(int2lab), lr_scheduler, tensorboard]

In [60]:
history = model.fit(x=train_tokens,
                    y=train_labels,
                    validation_data=(val_tokens, val_labels),
                    batch_size=64, epochs=epochs, verbose=1, callbacks=callbacks)

2020-01-07 12:28:25,193 From /home/gpuadmin/Candice/scripts/venv/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-01-07 12:28:26,004 From /home/gpuadmin/Candice/scripts/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

2020-01-07 12:28:26,024 From /home/gpuadmin/Candice/scripts/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 17433 samples, validate on 1937 samples


2020-01-07 12:28:26,343 From /home/gpuadmin/Candice/scripts/venv/lib/python3.6/site-packages/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.



Epoch 1/100
17433/17433 [==============================] - 47s 3ms/step - loss: 0.2064 - acc: 0.9598 - val_loss: 0.0840 - val_acc: 0.9773
             precision    recall  f1-score   support

   with_INS     0.0000    0.0000    0.0000       595
     to_INS     0.0000    0.0000    0.0000       276
     in_INS     0.0000    0.0000    0.0000        90
      –_INS     0.0000    0.0000    0.0000       366
       _INS     0.0000    0.0000    0.0000       286
      —_INS     0.0000    0.0000    0.0000       151
  after_INS     0.0000    0.0000    0.0000       155
 –_INS_ONLY     0.0000    0.0000    0.0000        32
because_INS     0.0000    0.0000    0.0000       118
     of_INS     0.0000    0.0000    0.0000        40
of_INS_ONLY     0.0000    0.0000    0.0000        13
to_INS_ONLY     0.0000    0.0000    0.0000        12

  micro avg     0.0000    0.0000    0.0000      2134
  macro avg     0.0000    0.0000    0.0000      2134

Epoch 2/100
17433/17433 [==============================] - 44s 3

17433/17433 [==============================] - 45s 3ms/step - loss: 0.0425 - acc: 0.9875 - val_loss: 0.0442 - val_acc: 0.9874
             precision    recall  f1-score   support

   with_INS     0.9416    0.9479    0.9447       595
     to_INS     0.0000    0.0000    0.0000       276
     in_INS     0.0000    0.0000    0.0000        90
      –_INS     0.3500    0.1339    0.1937       366
       _INS     0.4375    0.8077    0.5676       286
      —_INS     0.5500    0.3642    0.4382       151
  after_INS     0.9295    0.9355    0.9325       155
 –_INS_ONLY     0.0000    0.0000    0.0000        32
because_INS     0.0000    0.0000    0.0000       118
     of_INS     0.0000    0.0000    0.0000        40
of_INS_ONLY     0.0000    0.0000    0.0000        13
to_INS_ONLY     0.0000    0.0000    0.0000        12

  micro avg     0.6850    0.4892    0.5708      2134
  macro avg     0.4876    0.4892    0.4714      2134

Epoch 11/100
17433/17433 [==============================] - 45s 3ms/step - l

...

17433/17433 [==============================] - 45s 3ms/step - loss: 0.0305 - acc: 0.9902 - val_loss: 0.0345 - val_acc: 0.9900
             precision    recall  f1-score   support

   with_INS     0.9328    0.9563    0.9444       595
     to_INS     0.4966    0.2681    0.3482       276
     in_INS     0.6667    0.4000    0.5000        90
      –_INS     0.5286    0.4044    0.4582       366
       _INS     0.5520    0.7238    0.6263       286
      —_INS     0.6806    0.8609    0.7602       151
  after_INS     0.9308    0.9548    0.9427       155
 –_INS_ONLY     0.0000    0.0000    0.0000        32
because_INS     0.8594    0.4661    0.6044       118
     of_INS     0.0000    0.0000    0.0000        40
of_INS_ONLY     0.0000    0.0000    0.0000        13
to_INS_ONLY     0.0000    0.0000    0.0000        12

  micro avg     0.7264    0.6406    0.6808      2134
  macro avg     0.6804    0.6406    0.6477      2134

Epoch 65/100
17433/17433 [==============================] - 45s 3ms/step - l

17433/17433 [==============================] - 45s 3ms/step - loss: 0.0302 - acc: 0.9903 - val_loss: 0.0345 - val_acc: 0.9899
             precision    recall  f1-score   support

   with_INS     0.9375    0.9580    0.9476       595
     to_INS     0.6000    0.1848    0.2825       276
     in_INS     0.6939    0.3778    0.4892        90
      –_INS     0.4583    0.4809    0.4693       366
       _INS     0.5448    0.8287    0.6574       286
      —_INS     0.6753    0.8675    0.7594       151
  after_INS     0.9313    0.9613    0.9460       155
 –_INS_ONLY     0.0000    0.0000    0.0000        32
because_INS     0.8052    0.5254    0.6359       118
     of_INS     0.0000    0.0000    0.0000        40
of_INS_ONLY     0.0000    0.0000    0.0000        13
to_INS_ONLY     0.0000    0.0000    0.0000        12

  micro avg     0.7078    0.6607    0.6835      2134
  macro avg     0.6798    0.6607    0.6476      2134

Epoch 74/100
17433/17433 [==============================] - 45s 3ms/step - l

17433/17433 [==============================] - 45s 3ms/step - loss: 0.0300 - acc: 0.9903 - val_loss: 0.0349 - val_acc: 0.9894
             precision    recall  f1-score   support

   with_INS     0.9369    0.9479    0.9424       595
     to_INS     0.4203    0.3152    0.3602       276
     in_INS     0.6818    0.5000    0.5769        90
      –_INS     0.3947    0.3279    0.3582       366
       _INS     0.5428    0.8427    0.6603       286
      —_INS     0.6751    0.8808    0.7644       151
  after_INS     0.9308    0.9548    0.9427       155
 –_INS_ONLY     0.0000    0.0000    0.0000        32
because_INS     0.8750    0.4746    0.6154       118
     of_INS     0.0000    0.0000    0.0000        40
of_INS_ONLY     0.0000    0.0000    0.0000        13
to_INS_ONLY     0.0000    0.0000    0.0000        12

  micro avg     0.6823    0.6532    0.6675      2134
  macro avg     0.6485    0.6532    0.6402      2134

Epoch 83/100
17433/17433 [==============================] - 45s 3ms/step - l

17433/17433 [==============================] - 45s 3ms/step - loss: 0.0299 - acc: 0.9903 - val_loss: 0.0347 - val_acc: 0.9896
             precision    recall  f1-score   support

   with_INS     0.9359    0.9563    0.9460       595
     to_INS     0.4366    0.3370    0.3804       276
     in_INS     0.7091    0.4333    0.5379        90
      –_INS     0.4170    0.3087    0.3548       366
       _INS     0.5587    0.8322    0.6685       286
      —_INS     0.6768    0.8874    0.7679       151
  after_INS     0.9290    0.9290    0.9290       155
 –_INS_ONLY     0.0000    0.0000    0.0000        32
because_INS     0.8615    0.4746    0.6120       118
     of_INS     0.0000    0.0000    0.0000        40
of_INS_ONLY     0.0000    0.0000    0.0000        13
to_INS_ONLY     0.0000    0.0000    0.0000        12

  micro avg     0.6961    0.6495    0.6720      2134
  macro avg     0.6567    0.6495    0.6417      2134

Epoch 92/100
17433/17433 [==============================] - 44s 3ms/step - l

17433/17433 [==============================] - 45s 3ms/step - loss: 0.0298 - acc: 0.9904 - val_loss: 0.0344 - val_acc: 0.9898
             precision    recall  f1-score   support

   with_INS     0.9271    0.9613    0.9439       595
     to_INS     0.4828    0.3043    0.3733       276
     in_INS     0.6909    0.4222    0.5241        90
      –_INS     0.4295    0.3661    0.3953       366
       _INS     0.5556    0.8566    0.6740       286
      —_INS     0.6602    0.9007    0.7619       151
  after_INS     0.9255    0.9613    0.9430       155
 –_INS_ONLY     0.0000    0.0000    0.0000        32
because_INS     0.8529    0.4915    0.6237       118
     of_INS     0.0000    0.0000    0.0000        40
of_INS_ONLY     0.0000    0.0000    0.0000        13
to_INS_ONLY     0.0000    0.0000    0.0000        12

  micro avg     0.6962    0.6635    0.6795      2134
  macro avg     0.6593    0.6635    0.6486      2134



In [101]:
import json
from keras.models import load_model
from sklearn.preprocessing import OneHotEncoder
from joblib import dump, load

def save_json(obj, path):
    with open(path, 'w') as f:
        json.dump(obj, f)

def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

        
def save_model(path):
    
    model.save(path.joinpath("model_6_jan.h5"))
    
    save_json(vocab2int, path.joinpath('vocab2int.json'))
    save_json(int2vocab, path.joinpath('int2vocab.json'))
    save_json(lab2int, path.joinpath('lab2int.json'))
    save_json(int2lab, path.joinpath('int2lab.json'))
    dump(label_encoder, path.joinpath('label_encoder.pkl'))
    
def load_keras_model(path):
    
    model = load_model(path.joinpath("model.h5"))
    
    vocab2int = load_json(path.joinpath('vocab2int.json'))
    int2vocab = load_json(path.joinpath('int2vocab.json'))
    lab2int = load_json(path.joinpath('lab2int.json'))
    int2lab = load_json(path.joinpath('int2lab.json'))
    
    label_encoder = load(path.joinpath('label_encoder.pkl'))
    
    return {'model': model,
           'vocab2int':vocab2int,
           'int2vocab': int2vocab,
           'lab2int': lab2int,
           'int2lab': int2lab,
           'label_encoder': label_encoder}

    

In [102]:
%%time
from pathlib import Path
path = Path('/home/gpuadmin/Candice/scripts/temp/PREP_NER')
save_model(path)

CPU times: user 1.26 s, sys: 11.3 ms, total: 1.27 s
Wall time: 1.26 s


In [52]:
%time loaded_model = load_keras_model(path)

CPU times: user 2.71 s, sys: 4.69 s, total: 7.39 s
Wall time: 1.51 s


In [53]:
loaded_model.keys()

dict_keys(['model', 'vocab2int', 'int2vocab', 'lab2int', 'int2lab', 'label_encoder'])